In [27]:
import os

machine_path = os.getcwd()
user_root_dir = '/'.join(machine_path.split('/')[:-2])

import torch
import sys

# from torchvision import datasets, models, transforms

from PIL import Image
import torchvision

sys.path.append('%s/training_scaffold_own/res/'%user_root_dir)
from models.models import get_model
from loader.loader import get_loader
import random
import pickle

# Learning rate parameters
BASE_LR = 0.001
EPOCH_DECAY = 30 # number of epochs after which the Learning rate is decayed exponentially.
DECAY_WEIGHT = 0.1 # factor by which the learning rate is reduced.

# DATASET INFO
NUM_CLASSES = 2 # set the number of classes in your dataset
DATA_DIR = '../data/' # to run with the sample dataset, just set to 'hymenoptera_data'

# DATALOADER PROPERTIES
BATCH_SIZE = 10 # Set as high as possible. If you keep it too high, you'll get an out of memory error.


### GPU SETTINGS
CUDA_DEVICE = 0 # Enter device ID of your gpu if you want to run on gpu. Otherwise neglect.
GPU_MODE = 1 # set to 1 if want to run on gpu.

MODEL_ARCH = 'transformer'

from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections
from collections import OrderedDict
from tqdm import tqdm

In [2]:
IMAGENET_TRASNFORM = False

In [3]:
# DATASET_NAME = 'image_train_v3_shapenet_fixed_lighting_10_models'
DATASET_NAME = 'image_train_v7_shapenet_40'
# DATASET_NAME = 'image_train_v7_less_radius'
# DATASET_NAME = 'image_captured_data_unseen_3d_model_5000'
# DATASET_NAME = 'image_train_v3_symmetric_bias_6_conditions_camera'
# DATASET_NAME = 'image_train_v5_shapenet_zero_removed'
# DATASET_NAME = 'image_train_v7_shapenet_unseen_10'
# DATASET_NAME = 'image_test_v7_cma_resnet18_v7'
# DATASET_NAME = 'image_train_v7_clustered'
NUM_EPOCHS = 2
BATCH_SIZE = 75

# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                  std=[0.229, 0.224, 0.225])
# imagenet_trainsforms = {}
# imagenet_trainsforms['train'] = transforms.Compose([
#     transforms.RandomSizedCrop(224),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     normalize,
# ])

# imagenet_trainsforms['test'] = transforms.Compose([
#     transforms.Scale(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     normalize,
# ])

transforms_without_crop = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
    ])
transforms_with_crop = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.RandomResizedCrop(224),
    ])

if DATASET_NAME == 'imagenet' or IMAGENET_TRASNFORM:
    image_transform = {}
    image_transform['train'] = imagenet_trainsforms['train']
    image_transform['test'] = imagenet_trainsforms['test']
    image_transform['leftover'] = imagenet_trainsforms['test']
    image_transform['leftover_train'] = imagenet_trainsforms['test']
else:
    image_transform = {}
    image_transform['train'] = transforms_without_crop
    image_transform['test'] = transforms_without_crop
    image_transform['leftover'] = transforms_without_crop
GPU = 1

NUM_CLASSES = 55

In [4]:
loader_new = get_loader('multi_attribute_loader_file_list_shapenet')
if 'smadan' in user_root_dir:
    file_list_root = '%s/dataset_lists_openmind'%user_root_dir
elif 'spandan' in user_root_dir:
    file_list_root = "%s/dataset_lists_fasrc"%user_root_dir


att_path = '%s/differentiable_graphics_ml/training_models/shapenet_id_to_class_num.p'%user_root_dir
shuffles = {'train':True,'val':True,'test':True,'seen_test':False,'leftover':False,'leftover_train': False}

In [5]:
# os.path.isfile(file_lists['leftover'])

In [6]:
################ GET FROM USER CONFIG - TODO #####################
file_lists = {}
dsets = {}
dset_loaders = {}
dset_sizes = {}
for phase in ['test']:
    file_lists[phase] = "%s/%s_list_%s.txt"%(file_list_root,phase,DATASET_NAME)
    dsets[phase] = loader_new(file_lists[phase],att_path, image_transform[phase])
    dset_loaders[phase] = torch.utils.data.DataLoader(dsets[phase], batch_size=BATCH_SIZE, shuffle = shuffles[phase], num_workers=0,drop_last=True)
    dset_sizes[phase] = len(dsets[phase])

In [7]:
os.listdir('../training_models/saved_models/')

['fine_tuned_best_model_r18_1ep.pt',
 'simple_cnn_fine_tuned.pt',
 'simple_cnn_fine_tuned_2.pt',
 'alexnet.pt',
 'wandb_test.pt',
 'alexnet_20_models.pt',
 'resnet_20_models.pt',
 'resnet_20_models_pretrained.pt',
 'resnet_10_models.pt',
 'resnet_10_models_0001.pt',
 'resnet_10_models_00001.pt',
 'resnet_10_models_000001.pt',
 'resnet_10_models_00001_2',
 'resnet_10_models_00001_3.pt',
 'resnet_10_models_00001_3_final.pt',
 'alexnet_10_models_00001_3.pt',
 'simple_cnn_10_models_00001_3.pt',
 'alexnet_10_models_00001_3_final.pt',
 'simple_cnn_10_models_00001_3_final.pt',
 'lenet_shapenet.pt',
 'lenet_shapenet_200_epochs.pt',
 'lenet_shapenet_final.pt',
 'lenet_shapenet_200_epochs_final.pt',
 'TEST_1.pt',
 'alexnet_less_wide_10_models_00001.pt',
 'alexnet_less_wide_10_models_00001_final.pt',
 'alexnet_least_wide_10_models.pt',
 'alexnet_least_wide_10_models_final.pt',
 'alexnet_least_wide_train_v3.pt',
 'alexnet_least_wide_train_v3_final.pt',
 'resnet_train_v3.pt',
 'densenet_train_v3.pt

In [8]:
# model_path = '../training_models/saved_models/resnet18_train_v3_fixed_lighting.pt'
# model_path = '../training_models/saved_models/resnet18_v7_normalized.pt'
# model_path = '../training_models/saved_models/resnet18_train_v7_controlled_cluster_final.pt'
# model_path = '../training_models/saved_models/resnet18_captured_data_unseen_3d_model_5000_final.pt'
# model_path = '../training_models/saved_models/resnet18_v3_symmetric_bias_6_conditions_camera.pt'
# model_path = '../training_models/saved_models/resnet18_antialiased_v7_40_normalized.pt'
# model_path = '../training_models/saved_models/resnet18_v7_4'
# model_path = '../training_models/saved_models/resnet18_v7_fine_tuned_less_frozen.pt'
# model_path = '../training_models/saved_models/resnet18_v7_subsampled_normalized_final.pt'
model_path = '../training_models/saved_models/resnet18_v7_40_truly_shift_invariant_normalized.pt'

loaded_model = torch.load(model_path)
loaded_model.cuda();

In [9]:
if isinstance(loaded_model, torch.nn.DataParallel):
    loaded_model = loaded_model.module

In [10]:
model_name = model_path.split('/')[-1].split('.p')[0]

In [11]:
# loaded_model.eval();

In [12]:
BATCH_SIZE

75

In [13]:
model_name

'resnet18_v7_40_truly_shift_invariant_normalized'

In [14]:
all_corrects = 0
total = 0
incorrect_predicted_images = []
all_paths = []
for phase in ['test']:
    for data in tqdm(dset_loaders[phase]):
        inputs, labels, image_paths = data
        inputs = inputs.float().cuda()
#         inputs = inputs.type(torch.FloatTensor).cuda()
        labels = labels.long().cuda()
        if 'v7' in model_path:
            im_means = torch.mean(inputs.view(BATCH_SIZE, -1),dim=1).unsqueeze(1).unsqueeze(1).unsqueeze(1)
            im_stds = torch.std(inputs.view(BATCH_SIZE, -1),dim=1).unsqueeze(1).unsqueeze(1).unsqueeze(1)
            inputs = (inputs - im_means)/im_stds

        outputs = loaded_model(inputs)
        preds = torch.argmax(outputs,dim=1)
        corrects = torch.sum(preds == labels).item()
        all_corrects += corrects
        total += len(preds)
        all_paths.extend(image_paths)
        incorrect_predicted_images.extend([image_paths[i] for i in torch.where(preds!=labels)[0]])

100%|██████████| 525/525 [04:23<00:00,  1.99it/s]


In [15]:
len(incorrect_predicted_images)

7855

In [16]:
len(dsets['test'])

39399

In [17]:
acc = all_corrects/total

In [25]:
acc

0.8005079365079365

In [19]:
# model_predictions = {}
# model_predictions['correct'] = [i for i in all_paths if i not in incorrect_predicted_images]
# model_predictions['incorrect'] = incorrect_predicted_images

In [20]:
# len(model_predictions['correct'])

In [21]:
# len(model_predictions['incorrect'])

In [22]:
# len(model_predictions['correct'])/len(all_paths)

In [23]:
# '/om5/user/smadan/differentiable_graphics_ml/data/train_v7_shapenet/%s_v7_test.p'%model_name

In [24]:
# with open('/om5/user/smadan/differentiable_graphics_ml/data/train_v7_shapenet/%s_v7_test.p'%model_name, 'wb') as F:
#     pickle.dump(model_predictions, F)

In [26]:
with open('/om5/user/smadan/differentiable_graphics_ml/data/train_v7_shapenet/resnet18_truly_shift_invariant_incorrect_predictions.p', 'wb') as F:
    pickle.dump(incorrect_predicted_images, F)    

In [25]:
# folder = '/om5/user/smadan/differentiable_graphics_ml/data/train_v7_less_radius/02691156/images/'

In [172]:
# all_ims = os.listdir(folder)

In [173]:
# count = 0
# for im in sorted(all_ims):
#     plt.imshow(Image.open("%s/%s"%(folder, im)))
#     plt.axis('off')
#     plt.show()
#     count += 1
#     if count == 10:
#         break